# history dumps import to mariadb

#### Summary of notebook

Imported the history dumps to mariadb, each dump as a separate table  

- Used sqlalchemy/python as much as possible for easy replicability of process
    - failed so-far to do ```LOAD DATA LOCAL INFILE``` in python
    - did it in CLI with scripts generated and documented in this notebook
- Tracked and monitored warnings during ```LOAD DATA LOCAL INFILE```
    - adjusted the schema and repeated

## Generate MySQL queries

#### imports

##### libraries, datapath, metadata

In [1]:
import os, re, pyperclip, pandas as pd, numpy as np
from datetime import datetime as dt

datapath = '../data/raw/jawiki/dumps_unzipped/'

mtdt = pd.read_csv('../references/metadata/history.tsv', sep='\t')
# print(mtdt.col_name.tolist())

##### list of years for history dumpfiles ```years```

In [2]:
years = list(range(2013,2022))

### code to make queries

#### make query list ```drop_table_query_list```

In [3]:
drop_table_query_list = ['DROP TABLE IF EXISTS ' + 't' + str(year) + ';' for year in years]

#### make query list ```create_table_query_list```

##### store the query in variable

In [279]:
def create_table(year):
    create_table_1 = '''CREATE TABLE '''
    create_table_2 = ''' (  wiki_db VARCHAR(20) NOT NULL, event_entity VARCHAR(20) NOT NULL, event_type VARCHAR(20) NOT NULL, event_timestamp VARCHAR(30) NOT NULL, event_comment TEXT, event_user_id BIGINT, event_user_text_historical TEXT, event_user_text TEXT, event_user_blocks_historical VARCHAR(30), event_user_blocks VARCHAR(60), event_user_groups_historical TEXT, event_user_groups TEXT, event_user_is_bot_by_historical VARCHAR(10), event_user_is_bot_by VARCHAR(10), event_user_is_created_by_self VARCHAR(10),  event_user_is_created_by_system VARCHAR(10),  event_user_is_created_by_peer VARCHAR(10),  event_user_is_anonymous VARCHAR(10),  event_user_registration_timestamp VARCHAR(30), event_user_creation_timestamp VARCHAR(30), event_user_first_edit_timestamp VARCHAR(30), event_user_revision_count BIGINT, event_user_seconds_since_previous_revision BIGINT, page_id BIGINT, page_title_historical TEXT, page_title TEXT, page_namespace_historical INT, page_namespace_is_content_historical VARCHAR(10),  page_namespace INT, page_namespace_is_content VARCHAR(10),  page_is_redirect VARCHAR(20),  page_is_deleted VARCHAR(20),  page_creation_timestamp VARCHAR(30), page_first_edit_timestamp VARCHAR(30), page_revision_count BIGINT, page_seconds_since_previous_revision BIGINT, user_id BIGINT, user_text_historical VARCHAR(100), user_text VARCHAR(100), user_blocks_historical VARCHAR(50), user_blocks VARCHAR(50), user_groups_historical VARCHAR(50), user_groups VARCHAR(50), user_is_bot_by_historical VARCHAR(50), user_is_bot_by VARCHAR(50), user_is_created_by_self VARCHAR(50),  user_is_created_by_system VARCHAR(50),  user_is_created_by_peer VARCHAR(50),  user_is_anonymous VARCHAR(10),  user_registration_timestamp VARCHAR(30), user_creation_timestamp VARCHAR(30), user_first_edit_timestamp VARCHAR(30), revision_id BIGINT, revision_parent_id BIGINT, revision_minor_edit VARCHAR(10),  revision_deleted_parts VARCHAR(30), revision_deleted_parts_are_suppressed VARCHAR(10),  revision_text_bytes BIGINT, revision_text_bytes_diff BIGINT, revision_text_sha1 VARCHAR(50), revision_content_model VARCHAR(10),  revision_content_format VARCHAR(10),  revision_is_deleted_by_page_deletion VARCHAR(10),  revision_deleted_by_page_deletion_timestamp VARCHAR(30), revision_is_identity_reverted VARCHAR(10),  revision_first_identity_reverting_revision_id BIGINT, revision_seconds_to_identity_revert BIGINT, revision_is_identity_revert VARCHAR(10),  revision_is_from_before_page_creation VARCHAR(10),  revision_tags TEXT, row_id BIGINT NOT NULL AUTO_INCREMENT, PRIMARY KEY (row_id) );'''
    return create_table_1 + 't' + str(year) + create_table_2

create_table_query_list = [create_table(year) for year in years]

##### formatted markdown of create table query

```sql
CREATE TABLE ja_hist (  wiki_db VARCHAR(20) NOT NULL, event_entity VARCHAR(20) NOT NULL, event_type VARCHAR(20) NOT NULL, event_timestamp VARCHAR(30) NOT NULL, event_comment TEXT, event_user_id BIGINT, event_user_text_historical TEXT, event_user_text TEXT, event_user_blocks_historical VARCHAR(30), event_user_blocks VARCHAR(60), event_user_groups_historical TEXT, event_user_groups TEXT, event_user_is_bot_by_historical VARCHAR(10), event_user_is_bot_by VARCHAR(10), event_user_is_created_by_self VARCHAR(10),  event_user_is_created_by_system VARCHAR(10),  event_user_is_created_by_peer VARCHAR(10),  event_user_is_anonymous VARCHAR(10),  event_user_registration_timestamp VARCHAR(30), event_user_creation_timestamp VARCHAR(30), event_user_first_edit_timestamp VARCHAR(30), event_user_revision_count BIGINT, event_user_seconds_since_previous_revision BIGINT, page_id BIGINT, page_title_historical TEXT, page_title TEXT, page_namespace_historical INT, page_namespace_is_content_historical VARCHAR(10),  page_namespace INT, page_namespace_is_content VARCHAR(10),  page_is_redirect VARCHAR(20),  page_is_deleted VARCHAR(20),  page_creation_timestamp VARCHAR(30), page_first_edit_timestamp VARCHAR(30), page_revision_count BIGINT, page_seconds_since_previous_revision BIGINT, user_id BIGINT, user_text_historical VARCHAR(100), user_text VARCHAR(100), user_blocks_historical VARCHAR(50), user_blocks VARCHAR(50), user_groups_historical VARCHAR(50), user_groups VARCHAR(50), user_is_bot_by_historical VARCHAR(50), user_is_bot_by VARCHAR(50), user_is_created_by_self VARCHAR(50),  user_is_created_by_system VARCHAR(50),  user_is_created_by_peer VARCHAR(50),  user_is_anonymous VARCHAR(10),  user_registration_timestamp VARCHAR(30), user_creation_timestamp VARCHAR(30), user_first_edit_timestamp VARCHAR(30), revision_id BIGINT, revision_parent_id BIGINT, revision_minor_edit VARCHAR(10),  revision_deleted_parts VARCHAR(30), revision_deleted_parts_are_suppressed VARCHAR(10),  revision_text_bytes BIGINT, revision_text_bytes_diff BIGINT, revision_text_sha1 VARCHAR(50), revision_content_model VARCHAR(10),  revision_content_format VARCHAR(10),  revision_is_deleted_by_page_deletion VARCHAR(10),  revision_deleted_by_page_deletion_timestamp VARCHAR(30), revision_is_identity_reverted VARCHAR(10),  revision_first_identity_reverting_revision_id BIGINT, revision_seconds_to_identity_revert BIGINT, revision_is_identity_revert VARCHAR(10),  revision_is_from_before_page_creation VARCHAR(10),  revision_tags TEXT, row_id BIGINT NOT NULL AUTO_INCREMENT, PRIMARY KEY (row_id) );
```

#### make query list ```load_data_query_list```

*(list of multi-queries to paste into MySQL CLI)*

##### store multi-queries in list/array variable

In [5]:
colnames = ['wiki_db', 'event_entity', 'event_type', 'event_timestamp', 'event_comment', 'event_user_id', 'event_user_text_historical', 'event_user_text', 'event_user_blocks_historical', 'event_user_blocks', 'event_user_groups_historical', 'event_user_groups', 'event_user_is_bot_by_historical', 'event_user_is_bot_by', 'event_user_is_created_by_self', 'event_user_is_created_by_system', 'event_user_is_created_by_peer', 'event_user_is_anonymous', 'event_user_registration_timestamp', 'event_user_creation_timestamp', 'event_user_first_edit_timestamp', 'event_user_revision_count', 'event_user_seconds_since_previous_revision', 'page_id', 'page_title_historical', 'page_title', 'page_namespace_historical', 'page_namespace_is_content_historical', 'page_namespace', 'page_namespace_is_content', 'page_is_redirect', 'page_is_deleted', 'page_creation_timestamp', 'page_first_edit_timestamp', 'page_revision_count', 'page_seconds_since_previous_revision', 'user_id', 'user_text_historical', 'user_text', 'user_blocks_historical', 'user_blocks', 'user_groups_historical', 'user_groups', 'user_is_bot_by_historical', 'user_is_bot_by', 'user_is_created_by_self', 'user_is_created_by_system', 'user_is_created_by_peer', 'user_is_anonymous', 'user_registration_timestamp', 'user_creation_timestamp', 'user_first_edit_timestamp', 'revision_id', 'revision_parent_id', 'revision_minor_edit', 'revision_deleted_parts', 'revision_deleted_parts_are_suppressed', 'revision_text_bytes', 'revision_text_bytes_diff', 'revision_text_sha1', 'revision_content_model', 'revision_content_format', 'revision_is_deleted_by_page_deletion', 'revision_deleted_by_page_deletion_timestamp', 'revision_is_identity_reverted', 'revision_first_identity_reverting_revision_id', 'revision_seconds_to_identity_revert', 'revision_is_identity_revert', 'revision_is_from_before_page_creation', 'revision_tags', 'row_id']

In [6]:
def get_tsvpath(year):
    return f'/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.{str(year)}.tsv'

In [141]:
load_data_query_list = ([
r"""
LOAD DATA LOCAL INFILE """ + f"'{get_tsvpath(year)}' " + 
r"""
    INTO TABLE t""" + str(year) + r""" 
    CHARACTER SET utf8mb4 
    FIELDS TERMINATED BY '\t' 
    LINES TERMINATED BY '\n'

""" + '(@' + ',@'.join(colnames) + ')' +  # columns as @variables
r"""

SET
"""
+ ', '.join([f"{i} = NULLIF(@{i},'')" for i in colnames]) + ';' # recieve empty strings to NULL values
+ f"""

tee {'load_data_warnings/t' + str(year)}.log;
show warnings;
notee;"""

for year in years
])

##### previously-working text of load data local infile

```sql
LOAD DATA LOCAL INFILE '/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.2014.tsv' 
    INTO TABLE ja_hist  
    CHARACTER SET utf8mb4 
    FIELDS TERMINATED BY '\t' 
    LINES TERMINATED BY '\n' 
    (@wiki_db,@event_entity,@event_type,@event_timestamp,@event_comment,@event_user_id,@event_user_text_historical,@event_user_text,@event_user_blocks_historical,@event_user_blocks,@event_user_groups_historical,@event_user_groups,@event_user_is_bot_by_historical,@event_user_is_bot_by,@event_user_is_created_by_self,@event_user_is_created_by_system,@event_user_is_created_by_peer,@event_user_is_anonymous,@event_user_registration_timestamp,@event_user_creation_timestamp,@event_user_first_edit_timestamp,@event_user_revision_count,@event_user_seconds_since_previous_revision,@page_id,@page_title_historical,@page_title,@page_namespace_historical,@page_namespace_is_content_historical,@page_namespace,@page_namespace_is_content,@page_is_redirect,@page_is_deleted,@page_creation_timestamp,@page_first_edit_timestamp,@page_revision_count,@page_seconds_since_previous_revision,@user_id,@user_text_historical,@user_text,@user_blocks_historical,@user_blocks,@user_groups_historical,@user_groups,@user_is_bot_by_historical,@user_is_bot_by,@user_is_created_by_self,@user_is_created_by_system,@user_is_created_by_peer,@user_is_anonymous,@user_registration_timestamp,@user_creation_timestamp,@user_first_edit_timestamp,@revision_id,@revision_parent_id,@revision_minor_edit,@revision_deleted_parts,@revision_deleted_parts_are_suppressed,@revision_text_bytes,@revision_text_bytes_diff,@revision_text_sha1,@revision_content_model,@revision_content_format,@revision_is_deleted_by_page_deletion,@revision_deleted_by_page_deletion_timestamp,@revision_is_identity_reverted,@revision_first_identity_reverting_revision_id,@revision_seconds_to_identity_revert,@revision_is_identity_revert,@revision_is_from_before_page_creation,@revision_tags,@row_id) 

SET wiki_db = NULLIF(@wiki_db,''), event_entity = NULLIF(@event_entity,''), event_type = NULLIF(@event_type,''), event_timestamp = NULLIF(@event_timestamp,''), event_comment = NULLIF(@event_comment,''), event_user_id = NULLIF(@event_user_id,''), event_user_text_historical = NULLIF(@event_user_text_historical,''), event_user_text = NULLIF(@event_user_text,''), event_user_blocks_historical = NULLIF(@event_user_blocks_historical,''), event_user_blocks = NULLIF(@event_user_blocks,''), event_user_groups_historical = NULLIF(@event_user_groups_historical,''), event_user_groups = NULLIF(@event_user_groups,''), event_user_is_bot_by_historical = NULLIF(@event_user_is_bot_by_historical,''), event_user_is_bot_by = NULLIF(@event_user_is_bot_by,''), event_user_is_created_by_self = NULLIF(@event_user_is_created_by_self,''), event_user_is_created_by_system = NULLIF(@event_user_is_created_by_system,''), event_user_is_created_by_peer = NULLIF(@event_user_is_created_by_peer,''), event_user_is_anonymous = NULLIF(@event_user_is_anonymous,''), event_user_registration_timestamp = NULLIF(@event_user_registration_timestamp,''), event_user_creation_timestamp = NULLIF(@event_user_creation_timestamp,''), event_user_first_edit_timestamp = NULLIF(@event_user_first_edit_timestamp,''), event_user_revision_count = NULLIF(@event_user_revision_count,''), event_user_seconds_since_previous_revision = NULLIF(@event_user_seconds_since_previous_revision,''), page_id = NULLIF(@page_id,''), page_title_historical = NULLIF(@page_title_historical,''), page_title = NULLIF(@page_title,''), page_namespace_historical = NULLIF(@page_namespace_historical,''), page_namespace_is_content_historical = NULLIF(@page_namespace_is_content_historical,''), page_namespace = NULLIF(@page_namespace,''), page_namespace_is_content = NULLIF(@page_namespace_is_content,''), page_is_redirect = NULLIF(@page_is_redirect,''), page_is_deleted = NULLIF(@page_is_deleted,''), page_creation_timestamp = NULLIF(@page_creation_timestamp,''), page_first_edit_timestamp = NULLIF(@page_first_edit_timestamp,''), page_revision_count = NULLIF(@page_revision_count,''), page_seconds_since_previous_revision = NULLIF(@page_seconds_since_previous_revision,''), user_id = NULLIF(@user_id,''), user_text_historical = NULLIF(@user_text_historical,''), user_text = NULLIF(@user_text,''), user_blocks_historical = NULLIF(@user_blocks_historical,''), user_blocks = NULLIF(@user_blocks,''), user_groups_historical = NULLIF(@user_groups_historical,''), user_groups = NULLIF(@user_groups,''), user_is_bot_by_historical = NULLIF(@user_is_bot_by_historical,''), user_is_bot_by = NULLIF(@user_is_bot_by,''), user_is_created_by_self = NULLIF(@user_is_created_by_self,''), user_is_created_by_system = NULLIF(@user_is_created_by_system,''), user_is_created_by_peer = NULLIF(@user_is_created_by_peer,''), user_is_anonymous = NULLIF(@user_is_anonymous,''), user_registration_timestamp = NULLIF(@user_registration_timestamp,''), user_creation_timestamp = NULLIF(@user_creation_timestamp,''), user_first_edit_timestamp = NULLIF(@user_first_edit_timestamp,''), revision_id = NULLIF(@revision_id,''), revision_parent_id = NULLIF(@revision_parent_id,''), revision_minor_edit = NULLIF(@revision_minor_edit,''), revision_deleted_parts = NULLIF(@revision_deleted_parts,''), revision_deleted_parts_are_suppressed = NULLIF(@revision_deleted_parts_are_suppressed,''), revision_text_bytes = NULLIF(@revision_text_bytes,''), revision_text_bytes_diff = NULLIF(@revision_text_bytes_diff,''), revision_text_sha1 = NULLIF(@revision_text_sha1,''), revision_content_model = NULLIF(@revision_content_model,''), revision_content_format = NULLIF(@revision_content_format,''), revision_is_deleted_by_page_deletion = NULLIF(@revision_is_deleted_by_page_deletion,''), revision_deleted_by_page_deletion_timestamp = NULLIF(@revision_deleted_by_page_deletion_timestamp,''), revision_is_identity_reverted = NULLIF(@revision_is_identity_reverted,''), revision_first_identity_reverting_revision_id = NULLIF(@revision_first_identity_reverting_revision_id,''), revision_seconds_to_identity_revert = NULLIF(@revision_seconds_to_identity_revert,''), revision_is_identity_revert = NULLIF(@revision_is_identity_revert,''), revision_is_from_before_page_creation = NULLIF(@revision_is_from_before_page_creation,''), revision_tags = NULLIF(@revision_tags,''), row_id = NULLIF(@row_id,'');
```

##### get query to clipboard for pasting into mysql CLI

In [8]:
gen = (i for i in range(len(years)))

In [9]:
i = next(gen)
i, years[i]

(0, 2013)

In [62]:
pyperclip.copy(load_data_query_list[i]) # multiquery

## Connect to SQLAlchemy

#### imports

In [13]:
import mysql.connector as mysql
import sqlalchemy

### login

#### initialize mariadb

```bash
# mysql --user=root --password=XXXXXXXX
mysql -p
```

#### get credentials into notebook

In [29]:
mysql_user = 'bhrdwj'
# mysql_pass = input(f'Enter the MySQL password for user {mysql_user}: ')

#### connect to database

In [32]:
host='localhost'; user=mysql_user; passwd=mysql_pass; dbname='jawiki';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine = sqlalchemy.create_engine(connection_str)
    conn = engine.connect()
except Exception as e:
    print('Database connection error - check creds')
    print(e)

#### get ```tablenames```

In [34]:
metadata = sqlalchemy.MetaData(conn)
metadata.reflect()
current_tables = list(metadata.tables.keys())
current_tables

['category', 'categorylinks', 'page']

## Load all tables to mariadb

### setup

#### define function ```jpr_sql```

##### jpr_sql

In [38]:
def jpr_sql(query, conn=conn):
    try: 
        a = conn.execute(query)
    except Exception as e:
        print('Printing execute error:')
        print(e)
        return None
    
    try:
        return a.fetchall()
    except Exception as e:
        print('Printing results error:')
        print(e)
        print('Printing result')
        return None

#### check connection

##### reconnect to database

In [381]:
host='localhost'; user=mysql_user; passwd=mysql_pass; dbname='jawiki';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine = sqlalchemy.create_engine(connection_str)
    conn = engine.connect()
except:
    print('Database connection error - check creds')

##### check it

In [355]:
current_tables = jpr_sql(f'show tables;')
current_tables = [i[0] for i in current_tables]
display(current_tables)

jpr_sql(f'select * from {current_tables[0]} limit 10;');
jpr_sql(f'desc {current_tables[0]};');

['category',
 'categorylinks',
 'page',
 't2013',
 't2014',
 't2015',
 't2016',
 't2017',
 't2018']

### babysit the load data loop

#### before starting

In [244]:
jpr_sql('set global max_error_count=100000;')

Printing execute error:
(mysql.connector.errors.ProgrammingError) 1227 (42000): Access denied; you need (at least one of) the SUPER privilege(s) for this operation
[SQL: set global max_error_count=100000;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [245]:
jpr_sql("""
SELECT VARIABLE_NAME, SESSION_VALUE, GLOBAL_VALUE FROM
INFORMATION_SCHEMA.SYSTEM_VARIABLES WHERE 
VARIABLE_NAME IN ('max_error_count');""")

[('MAX_ERROR_COUNT', '65535', '65535')]

In [295]:
gen = (i for i in range(len(years)))

#### loop

In [391]:
i = next(gen)

In [392]:
i, years[i]

(8, 2021)

In [393]:
jpr_sql(drop_table_query_list[i])

Printing results error:
This result object does not return rows. It has been closed automatically.
Printing result


In [394]:
jpr_sql(create_table_query_list[i])

Printing results error:
This result object does not return rows. It has been closed automatically.
Printing result


In [395]:
jpr_sql('show tables;')

[('category',),
 ('categorylinks',),
 ('page',),
 ('t2013',),
 ('t2014',),
 ('t2015',),
 ('t2016',),
 ('t2017',),
 ('t2018',),
 ('t2019',),
 ('t2020',),
 ('t2021',)]

In [396]:
pyperclip.copy(load_data_query_list[i]) # multiquery

*paste and run into mariadb cli*

#### check

In [389]:
years[i]

2020

In [382]:
jpr_sql('show tables;')

[('category',),
 ('categorylinks',),
 ('page',),
 ('t2013',),
 ('t2014',),
 ('t2015',),
 ('t2016',),
 ('t2017',),
 ('t2018',),
 ('t2019',),
 ('t2020',)]

In [388]:
jpr_sql(f'select page_id from t{years[i]} limit 10;')

[(4057210,),
 (4057210,),
 (4057210,),
 (1974801,),
 (4057211,),
 (4057211,),
 (4057211,),
 (3554070,),
 (4052485,),
 (1113014,)]

In [384]:
jpr_sql(f'desc t{years[i]};');

In [ ]:
jpr_sql(f'select count(*) from t{years[i]};')

## Restrict user bhrdwj to read-only access to raw data

### revoke

In [385]:
current_tables = jpr_sql(f'show tables;')
current_tables = [i[0] for i in current_tables]

In [386]:
restricted_tables = current_tables
restricted_user = 'bhrdwj'

In [401]:
print('\n'.join([f"REVOKE ALL ON jawiki.{table} FROM {user};" for table in current_tables]))

REVOKE ALL ON jawiki.category FROM bhrdwj;
REVOKE ALL ON jawiki.categorylinks FROM bhrdwj;
REVOKE ALL ON jawiki.page FROM bhrdwj;
REVOKE ALL ON jawiki.t2013 FROM bhrdwj;
REVOKE ALL ON jawiki.t2014 FROM bhrdwj;
REVOKE ALL ON jawiki.t2015 FROM bhrdwj;
REVOKE ALL ON jawiki.t2016 FROM bhrdwj;
REVOKE ALL ON jawiki.t2017 FROM bhrdwj;
REVOKE ALL ON jawiki.t2018 FROM bhrdwj;
REVOKE ALL ON jawiki.t2019 FROM bhrdwj;
REVOKE ALL ON jawiki.t2020 FROM bhrdwj;


### grant

In [397]:
'GRANT SELECT, SHOW VIEW, PROCESS, REPLICATION CLIENT ON *.* TO bhrdwj;'

'GRANT SELECT, SHOW VIEW, PROCESS, REPLICATION CLIENT ON *.* TO bhrdwj;'

## EDA for adjusting ```load data local infile``` settings

##### ```revision_minor_edit```

```sql
select revision_minor_edit, min(row_id), count(*) 
    from ja_hist 
    group by revision_minor_edit order by row_id;
```

##### ```user_is_anonymous```

```sql
select user_is_anonymous, min(row_id), count(*) 
    from ja_hist 
    group by user_is_anonymous order by row_id;
```

##### ```user_text_historical```

```sql
select user_text_historical, min(row_id), count(*) 
    from ja_hist 
    where user_text_historical regexp '^[0-9]*$' 
    group by user_text_historical order by row_id;
```

##### ```revision_content_model```

##### ```user_is_bot_by_historical```

##### ```page_revision_count```

# END